# VQE for Track Reconstruction

This is an introduction about the concepts of using VQE in the context of track reconstruction, more precisly solving a QUBO which represent track candidates. This notebook will provide you with information and tasks to consolidate your knowledge.

If something is unclear or if you just want to discuss something, don't forget:

**We are always happy to answer your questions and to help you at any stage of the project!**

## Checklist before starting to work on this noteboook

If you feel, that (parts of) the following statements is not true, let's schedule a brief zoom meeting to make them true :)

* I have knowlegde about the concepts of a preselection (from a previous notebook ;-))
    - doublect selection
    - triplet selection
* I have knowledge about the QUBO and it's link to track reconstruction
    - triplet interaction terms
    - triplet quality terms
    - energy evaluation



##  I. Framework Introduction
    a) preselection framework
          - task 1: using the preselection framework 

## II. VQE preparation of a basic example
    a) creating triplets from a tracking data file
    b) loading triplet list and creating QUBO matrix 
          - task 2: energy evaluation
    c) converting the QUBO to an Ising model
          - task 3: Ising representation
          
## III. Solving the QUBO
    a) Numpy Eigensolver exact solution
    b) solving the QUBO with ideal VQE
          - task 4: understanding the results part I
    c) solving the QUBO on a Fake Device
          - task 5: understanding the results part II
    d) solving the QUBO with VQE on a real quantum device

##  I. Framework Introduction

Before we can start with VQE, we would like to introduce a faster way of executing the preselection:

From now on you may use a part of our framework for the preselection. It's basically a more sophisticated version of the code in the track reconstruction basics notebook. If you pass the parameters correctly, a .npy file containing triplets is created. This file we can then use for the VQE approach.

The framework (or at least it's a small piece of of our framework) also creates some statistics.
In the following, we prepared a small example on how we perform the preselection with our framework. Click and enjoy!

## Task 1: using the preselection framework

    a) Execute the code below and look at the output files in the newly created folder. Can you explain what they show and how to interprete results! Let's discuss them!

    b) Schedule a meeting with us and let's talk about (a) and about the preselection and how you can access and tune it for your summer student project. 

In [1]:
%%capture
from preselection.x_plets import XpletCreatorLUXE
from pathlib import Path
import sys
import yaml

import os

# ---------------------------------------------------------------------------------------------------------------------- # 
configuration_file =  "preselection/configurations/LUXE.yaml"          # prepared configuration file
tracking_data_file = "example/example_event.csv"                       # prepared .csv tracking data file
output_folder = "example"                                              # storing results
# ---------------------------------------------------------------------------------------------------------------------- # 


# XPLetCreator manages the creation of doublets and triplets. The parameters are read from the config file.
with open(configuration_file, 'r') as f:
    config_file = yaml.safe_load(f)

# Creating a new folder which is named as the config file and stored inside the folder according to sys.argv[3]
new_folder = output_folder + "/" + tracking_data_file.split("/")[-1].split(".csv")[0] + "-" + \
             ".".join(configuration_file.split("/")[-1].split(".")[0:-1])

if Path(new_folder).is_dir():
    pass
else:
    os.mkdir(new_folder)

# Creating Xplet Object
x_plet_creator = XpletCreatorLUXE(tracking_data_file, config_file, new_folder)
x_plet_creator.make_simplified_x_plet_list()
x_plet_creator.set_triplet_coefficients()
x_plet_creator.write_info_file()
x_plet_creator.plot_and_save_statistics()

## II. VQE preparation of a basic example 

In this introduction you will start your VQE experience with a very basic example:

* 8 hits on the detector layers resulting in 2 tracks
* 2 tracks consist of 4 triplets in total, but in this example there are 5 triplets and it's on the algorithm and on you to decide which triplets to keep

You will use the following data, stemming from a Simplified Simulation:

|hit_ID| |x|                   |y|                      |z|   |layer| |particle_ID| |particle_energy|
|------| |-|                   |-|                      |-|   |-----| |-----------| |---------------|
|0|      |0.12103957869794268| |-0.0005543368424526175| |3.9| |1|     |0|           |7000.0|
|1|      |0.10253193453685608| |-0.0015610591253087892| |3.9| |1|     |1|           |8000.0|
|2|      |0.12784860784208985| |-0.0005494096126926273| |4.0| |2|     |0|           |7000.0|
|3|      |0.10828159267753819| |-0.0016004537865348845| |4.0| |2|     |1|           |8000.0|
|4|      |0.13464833759466250| |-0.0005370527994169504| |4.1| |3|     |0|           |7000.0|
|5|      |0.11403923270685501| |-0.0016270673269656798| |4.1| |3|     |1|           |8000.0|
|6|      |0.14144026879465033| |-0.0005494048105441511| |4.2| |4|     |0|           |7000.0|
|7|      |0.11978100416246684| |-0.0016458699843569216| |4.2| |4|     |1|           |8000.0|

A file containing these data is prepared. 

## a) Creating triplets from a tracking data file

With the code below, the data is loaded and processed. All parameters for the QUBO are set and saved into a .npy file which contains a list of triplet objects. Please have a look at the generated output statistic files!

In [2]:
%%capture
from preselection.x_plets import XpletCreatorLUXE
from pathlib import Path
import sys
import yaml

import os

# ---------------------------------------------------------------------------------------------------------------------- # 
configuration_file =  "preselection/configurations/intro.yaml"   # prepared configuration file
tracking_data_file = "example/example_VQE.csv"                         # prepared .csv tracking data file
output_folder = "example"                                              # storing results
# ---------------------------------------------------------------------------------------------------------------------- # 


# XPLetCreator manages the creation of doublets and triplets. The parameters are read from the config file.
with open(configuration_file, 'r') as f:
    config_file = yaml.safe_load(f)

# Creating a new folder which is named as the config file and stored inside the folder according to sys.argv[3]
new_folder = output_folder + "/" + tracking_data_file.split("/")[-1].split(".csv")[0] + "-" + \
             ".".join(configuration_file.split("/")[-1].split(".")[0:-1])

if Path(new_folder).is_dir():
    pass
else:
    os.mkdir(new_folder)

# Creating Xplet Object
x_plet_creator = XpletCreatorLUXE(tracking_data_file, config_file, new_folder)
x_plet_creator.make_simplified_x_plet_list()
x_plet_creator.set_triplet_coefficients()
x_plet_creator.write_info_file()
x_plet_creator.plot_and_save_statistics()

## b) Loading triplet list and creating QUBO matrix 

In [3]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML, clear_output

pd.set_option('precision', 2)

triplet_list = np.load("example/example_VQE-intro/triplet_list.npy", allow_pickle=True)
size = len(triplet_list) 


# QUBO matrix, a_i are values on the diagonal, b_ij for values off the diagonal
a_i = np.zeros(size)
b_ij = np.zeros((size, size))

# filling the QUBO matrix
for t in triplet_list:
    a_i[t.triplet_id] = t.quality
    for interaction in t.interactions.keys():
        if interaction > t.triplet_id:
            b_ij[t.triplet_id, interaction] = t.interactions[interaction]

qubo_matrix = np.diag(a_i) + b_ij   

print("QUBO Matrix (values rounded to the first decimal place):")
display(pd.DataFrame(qubo_matrix))

QUBO Matrix (values rounded to the first decimal place):


,0,1,2,3,4
0,-0.1,0.0,0.0,0.0,-1.0
1,0.0,-0.1,1.0,-1.0,0.0
2,0.0,0.0,0.1,0.0,-0.9
3,0.0,0.0,0.0,-0.1,0.0
4,0.0,0.0,0.0,0.0,-0.1


## Task 2: energy evaluation

    a) How is the energy of a QUBO with a given solution evaluated? 

    b) What is the energy of the system according to the matrix above if the solution is:
    
        i)   [1, 1, 1, 1, 1]
        ii)  [0, 0, 0, 0, 0]
        iii) [1, 1, 0, 1, 1]

## c) Creating the QUBO as QuadraticProgram

There are some substeps now. On the technical side, we have to create a binary variable for every triplet and merge the variables with the QUBO matrix to get our objective. One can check the output and find
    
    - the objective we want to minimise
    - the allowed range of the variables
    - the name of the variables
    
if everything makes sense, then proceed.

In [4]:
%%capture
from qiskit.optimization.problems import QuadraticProgram

In [5]:
qubo = QuadraticProgram()
for i in range(size):
    qubo.binary_var(name='x' + str(i))
qubo.minimize(linear=a_i, quadratic=b_ij)

print(qubo)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 0.099965482509 x0 - 0.100000000000 x1 + 0.100000000000 x2
      - 0.099847969457 x3 - 0.099937471079 x4 + [ - 2 x0*x4 + 2 x1*x2
      - 1.999958422656 x1*x3 - 1.800000000000 x2*x4 ]/2
Subject To

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1

Binaries
 x0 x1 x2 x3 x4
End



## c) Converting the QUBO to an Ising model

The Qubo has to be converted to an Ising model in order to make computation on a quantum computer possible.

In [6]:
%%capture
op, offset = qubo.to_ising()

In [7]:
display(pd.DataFrame(op))

,0
0,0.29998274125443714 * IIIIZ
1,0.04999480283204502 * IIIZI
2,-0.07499999999999998 * IIZII
3,0.29991878756069884 * IZIII
4,0.5249687355396694 * ZIIII
5,0.25 * IIZZI
6,-0.24999480283204503 * IZIZI
7,-0.25 * ZIIIZ
8,-0.225 * ZIZII


In [8]:
pd.options.display.max_columns = 10
pd.options.display.max_rows = 10
display(pd.DataFrame(op.to_matrix()))

,0,1,2,3,4,...,27,28,29,30,31
0,0.62+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,...,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j
1,0.00+0.00j,0.52+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,...,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j
2,0.00+0.00j,0.00+0.00j,0.52+0.00j,0.00+0.00j,0.00+0.00j,...,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j
3,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.42+0.00j,0.00+0.00j,...,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j
4,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.72+0.00j,...,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j
...,...,...,...,...,...,...,...,...,...,...,...
27,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,...,-1.77+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j
28,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,...,0.00+0.00j,-0.37+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j
29,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,...,0.00+0.00j,0.00+0.00j,-1.47+0.00j,0.00+0.00j,0.00+0.00j
30,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,0.00+0.00j,...,0.00+0.00j,0.00+0.00j,0.00+0.00j,-0.47+0.00j,0.00+0.00j


## Task 3: Ising representation

    a) Explain what is happening when calling:
        
        i)  display(pd.DataFrame(op))
        ii) display(pd.DataFrame(op.to_matrix()))
       
    b) Explain the outputs?
    
        i)   What means I, Z?
        ii)  What's with the size of the second output? (Looking for a certain keyword starting with a "T") 
        iii) Why do the values of the Ising representation differ from the QUBO?

##  III. QUBO solving

## a) Numpy Eigensolver exact solution

Before solving the with VQE we solve it analytically by evaluating the minimum Eigenvalue, thus finding the ground state of the QUBO / Ising.

In [9]:
%%capture
from qiskit.opflow.primitive_ops.pauli_sum_op import PauliSumOp
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer

ee = MinimumEigenOptimizer(NumPyMinimumEigensolver())
result = ee.solve(qubo).x

In [10]:
print('Result: ', result)
print('Energy: ', result.dot(qubo_matrix.dot(result)))

Result:  [1. 1. 0. 1. 1.]
Energy:  -2.399730134373701


## b) Solving the QUBO with VQE on ideal simulation

To solve the QUBO exactly some key ingredients are needed:

    - Hamiltonian in the form of an Ising / QUBO
    - a dedicated quantum circuit - called ansatz
        i)  TwoLocal
        ii) Custom Ansatz
        
    - optimizer:
        i)   COBYLA
        ii)  SPSA
        iii) NFT

    - backend:
        i)   an ideal simulation, 
        ii)  a fake device (noise modelled after a real quantum device)
        iii) real quantum device from IBM
        
We already defined our Hamiltonian, but since Qiskit is evolving fast sometimes a workaround is needed to utilize certain objects. In the following cell the Hamiltonian is just transformed into another representation, so that it is usable for VQE.
    

In [11]:
# qubo -> PauliSumOp for VQE
operator_list = []
for operator in op:
    aux_list = str(operator).split(' * ')
    t = tuple([aux_list[1], aux_list[0]])

    operator_list.append(t)

H = PauliSumOp.from_list(operator_list)
print(H)

0.29998274125443714 * IIIIZ
+ 0.04999480283204502 * IIIZI
- 0.07499999999999998 * IIZII
+ 0.29991878756069884 * IZIII
+ 0.5249687355396694 * ZIIII
+ 0.25 * IIZZI
- 0.24999480283204503 * IZIZI
- 0.25 * ZIIIZ
- 0.225 * ZIZII


The ansatz circuit has to be defined. In a previous notebook, you already learned how to build quantum circuits and how to use gates on qubits and also extract results in the form of a measurement. For track reconstruction we need as many qubits as there are triplet. So our quantum circuit has to be of size 5 for now. For track reconstruction algorithms with a huge number of tracks and thus a huge number of triplets, there are strategies to split the QUBO into smalller pieces. But for now we stick to a size-5 quantum circuit for our 5 triplets. 

There are many heuristics to build a quantum circuit suitable for various problem formulations. First we want to have a look at the TwoLoacl approach.

With

    ansatz = TwoLocal(num_qubits,
                      rotation_blocks,
                      entanglement_blocks,
                      reps,
                      entanglement)

a quantum circuit can be created where num_qubits is an integer number for the size of the quantum circuit, rotation_clocks takes a single string like 'ry' or 'rx' or a list ['rx', 'ry'] to enable the actual use of the variational principle,
entanglement_blocks takes any two or three qubit gate like 'cx' or a list of them ['cx'. 'sx'] to create entanglements, reps takes an integer number to create this number of repetitions of the before defined parametrized and entanglement gates. The entanglement structure is defined by the last parameter, 'linear', 'circular' and 'full' are common choices. More details in the documentation: https://qiskit.org/documentation/stable/0.24/stubs/qiskit.circuit.library.TwoLocal.html 

Let's start with a very simple example:

In [12]:
from qiskit.circuit.library import TwoLocal
from qiskit.tools.visualization import circuit_drawer

ansatz = TwoLocal(num_qubits=5, 
                  rotation_blocks='ry',
                  entanglement_blocks='cx', 
                  reps=1, 
                  entanglement='linear',
                  insert_barriers=True)
print(ansatz)

     ┌──────────┐ ░                      ░ ┌──────────┐
q_0: ┤ Ry(θ[0]) ├─░───■──────────────────░─┤ Ry(θ[5]) ├
     ├──────────┤ ░ ┌─┴─┐                ░ ├──────────┤
q_1: ┤ Ry(θ[1]) ├─░─┤ X ├──■─────────────░─┤ Ry(θ[6]) ├
     ├──────────┤ ░ └───┘┌─┴─┐           ░ ├──────────┤
q_2: ┤ Ry(θ[2]) ├─░──────┤ X ├──■────────░─┤ Ry(θ[7]) ├
     ├──────────┤ ░      └───┘┌─┴─┐      ░ ├──────────┤
q_3: ┤ Ry(θ[3]) ├─░───────────┤ X ├──■───░─┤ Ry(θ[8]) ├
     ├──────────┤ ░           └───┘┌─┴─┐ ░ ├──────────┤
q_4: ┤ Ry(θ[4]) ├─░────────────────┤ X ├─░─┤ Ry(θ[9]) ├
     └──────────┘ ░                └───┘ ░ └──────────┘


On the optimizer side we suggest to use the following three optimizers, but you can also add other optimizers. You would just need to look up the available ones on qiskit and add them to the following cells

In [13]:
from qiskit.algorithms.optimizers import COBYLA, SPSA, NFT

# optimizer = COBYLA(maxiter=4000)
# optimizer = SPSA(maxiter=4000)       # takes some time ...
optimizer = NFT(maxiter=4000)

The last ingredient is the quantum instance. As backend we take the 'qasm_simulator', an ideal simulation without noise.
The number of shots is the number of repetitions of the evaluation of the quantum circuit.

In [14]:
from qiskit import Aer
from qiskit.utils import QuantumInstance

In [15]:
quantum_instance = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), 
                                   shots=128)

Now everything can be put together and VQE can be finally used to solve the QUBO

In [16]:
from qiskit.algorithms import VQE

In [17]:
ideal_vqe = VQE(ansatz=ansatz,
                optimizer=optimizer,
                quantum_instance=quantum_instance)

ideal_result_vqe = ideal_vqe.compute_minimum_eigenvalue(H)

In [18]:
print('energy: ', ideal_result_vqe.eigenvalue.real)
print('result: ', ideal_result_vqe.eigenstate)

energy:  -1.7748598700188953
result:  {'01011': 0.0625, '11011': 0.998044963916957}


## Task 4: Understanding the results part I

    a) compare results of the Minimum Eigenoptimizer with the one of VQE and explain the results:
        i)   why do they differ in energy 
        ii)  why does VQE has many results and what do the values stand for 
        
        (Note: These results are ordered from last to first qubit! So it's not q0q1q2q3q4 but q4q3q2q1q0)
    
    b) in preparation for the next step: plot the probabilities(!) of the VQE results
    
    c) play around with optimizer and ansatz circuit try to find a combination that will result in the correct solution with 100% probability. Explain which parameters you altered and how you found the solution.


## c) Solving the QUBO with VQE on a Fake Device

Now let's get results on a Fake Device.

In [19]:
from qiskit.providers.aer import QasmSimulator
from qiskit.test.mock import FakeLima

def callback(eval_count, parameters, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []

quantum_instance = QuantumInstance(backend=QasmSimulator.from_backend(FakeLima()),
                                   shots=128,
                                   optimization_level=2)

vqe_fake_device = VQE(ansatz=ansatz,
                      optimizer=optimizer,
                      quantum_instance=quantum_instance,
                      callback=callback)

vqe_result_fake_device = vqe_fake_device.compute_minimum_eigenvalue(H)

'Evaluation: 1025, Energy: -1.4666800662050326, Std: 0.03291048286886297'

In [20]:
print('energy: ', vqe_result_fake_device.eigenvalue.real)
print('result: ', vqe_result_fake_device.eigenstate)

energy:  -1.4666800662050326
result:  {'00011': 0.08838834764831845, '01001': 0.0625, '01011': 0.3423265984407288, '10010': 0.08838834764831845, '10011': 0.21650635094610965, '11001': 0.21650635094610965, '11010': 0.0625, '11011': 0.8705242673240075, '11100': 0.0625, '11111': 0.0625}


## Task 5: Understanding the results part II

    a)  play around with optimizer and ansatz circuit again. Can you find a combination that will result in the correct solution with 100% probability. What's the problem here? Discuss!
    
    b) (Optional) There are topics within Quantum Computing called Error Mitigation and Error Correction. If you are interested about that let's schedule talk about it!

## d) Solving the QUBO with VQE on a real quantum device

Before you start this section:

    - Create an account at IBM at https://quantum-computing.ibm.com/. You need to have an API token in order to start computing on their devices. Don't worry, it's for free unless you want to pay for priority or better devices.

    - make sure you have chosen an optimal setup! It needs to be fast and give you a good result. Otherwise you wiill have to wait O(days) for your result, because you have no priority on the devices and have to share them with many other people.
    
    - after you have your results, please save them immediately. The print statements displays the results that are most important. A high resolution screenshot is appreciated to for starters ;-)

In [21]:
%%capture
from qiskit import IBMQ

In [22]:
# IBMQ.save_account("Here you have to put your API token")

In [ ]:
IBMQ.load_account()

In [ ]:
def callback(eval_count, parameters, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []

quantum_instance_ibmq = QuantumInstance(backend=IBMQ.get_provider(hub='ibm-q').get_backend('ibmq_quito'), 
                                        shots=128,
                                        optimization_level=2)
vqe_ibm = VQE(ansatz=ansatz,
              optimizer=optimizer,
              quantum_instance=quantum_instance_ibmq,
              callback=callback)



result_vqe_ibm = vqe_ibm.compute_minimum_eigenvalue(H)

In [ ]:
print('energy: ', result_vqe_ibm.eigenvalue.real)
print('result: ', result_vqe_ibm.eigenstate)

# Congratulations! 

# You completed your first track reconstruction task on a real quantum device. AWESOME!